# Modelado para un único modelo

Para el caso de un único modelo, la aplicación de los datos obtenidos se puede realizar de manera directa.

A continuación, se definen variables globales y las librerías que serán empleadas:

In [1]:
import os
from pathlib import Path
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import pickle
import numpy as np
from preprocess.preprocess import Preprocess
from confusion_matrix_pretty_print import print_confusion_matrix


# LBP_METHOD = 'default'
# LBP_METHOD = 'riu'
LBP_METHOD = 'riu2'
METHOD = 'get_pyramid_dataset'
# METHOD = 'get_datasets_by_scale'
HEIGHT = 608

En primer lugar se cargan las bases de datos:

In [2]:
df_train = pd.read_pickle('jupyter-media/train_train_' + LBP_METHOD + '_' + METHOD + '.pkl')
df_test = pd.read_pickle('jupyter-media/train_test_' + LBP_METHOD + '_' + METHOD + '.pkl')

Se divide la variable dependiente de las independientes:

In [3]:
y_train = df_train.loc[:, 'label']
y_test = df_test.loc[:, 'label']
df_train.drop(columns=['label'], inplace=True)
df_test.drop(columns=['label'], inplace=True)

Se entrena el modelo:

In [4]:
def init_clf_and_fit(df, y):
    classifier = GaussianNB()
    classifier.fit(df, y)
    return classifier

clf = init_clf_and_fit(df_train, y_train)

Y se realizan las predicciones y obtienen las métricas:

In [10]:
y_predicted = clf.predict(df_test)

print('Accuracy score: ' + str(accuracy_score(y_test, y_predicted)) + '\n')
print('F1 score: ' + str(f1_score(y_test, y_predicted)) + '\n')
confurion_matrix = confusion_matrix(y_test, y_predicted)
print('Confusion matrix:\n')
print_confusion_matrix(y_test, y_predicted)

Accuracy score: 0.6926954950697398

F1 score: 0.3522988452412991

Confusion matrix:

     t/p      0     1 
        0 830053 357492 
        1 61274 113888 


# Modelado para un modelo en cada escala

Se recogen los datos y se entrena un modelo para cada escala de la misma forma que en el caso previo:

In [11]:
METHOD = 'get_datasets_by_scale'

with open('jupyter-media/train_train_' + LBP_METHOD + '_' + METHOD + '.pkl', 'rb') as f:
    df_train_list = pickle.load(f)
clf_list = [init_clf_and_fit(df_train.iloc[:, :-1], df_train.iloc[:, -1]) for df_train in df_train_list]
with open('jupyter-media/train_test_' + LBP_METHOD + '_' + METHOD + '.pkl', 'rb') as f:
    df_test_list = pickle.load(f)

La aplicación de los modelos resulta algo más compleja. Se recogen las bases de datos correspondientes a cada escala y se aplican los modelos correspondientes. El resultado es un conjunto de predicciones de cada píxel a diferentes resoluciones, por lo que es preciso realizar un mapeo de los píxeles entre sí. Para ello se repiten los píxeles tantas veces como sea preciso para igualar el número de píxeles original de la imagen. Y posteriormente se aplica la máscara en su resolución original con la finalidad de conservar únicamente los píxeles relevantes. Las predicciones se ensamblan mediante un voto por mayoría, es decir se calcula la media de los valores y para aquellos superiores a 0.5 se etiqueta el píxel como perteneciente a un vaso sanguíneo.

Como vector de etiquetas reales se recoge el correspondiente a la dimensión original para cada imagen.

In [12]:
def ensemble_prediction(classifiers, dfs_test):
    # Calculating predictions
    y_predicted_list = [classifier.predict(test_set.iloc[:, :-1]).reshape(-1, 1)
                        for classifier, test_set in zip(classifiers, dfs_test['datasets'])]
    # Equaling the number of pixels
    y_predicted_list = [y_predicted_list[i].reshape(HEIGHT//(2 ** i), -1) for i in range(len(y_predicted_list))]
    y_predicted_list = [Preprocess.repeat_pixels(prediction, (2 ** i))
                        for i, prediction in enumerate(y_predicted_list)]
    # Mask application
    y_predicted_list = [prediction.ravel()[dfs_test['mask']].reshape(-1, 1) for prediction in y_predicted_list]
    # print(np.mean(np.concatenate(y_predicted_list, axis=1), axis=0))
    # Predictions
    y_predictions = np.mean(np.concatenate(y_predicted_list, axis=1), axis=1)
    # Actual values
    y_actual = np.array(dfs_test['datasets'][0].iloc[:, -1]).ravel()[dfs_test['mask']]
    return y_predictions, y_actual

In [13]:
    y_predicted_test_list = [ensemble_prediction(clf_list, dfs_test) for dfs_test in df_test_list]
    y_predicted = np.concatenate([y_predicted_test_pic[0] for y_predicted_test_pic in y_predicted_test_list])
    y_predicted = np.where(y_predicted > 0.5, 1, 0)
    y_test = np.concatenate([y_predicted_test_pic[1] for y_predicted_test_pic in y_predicted_test_list])

Las métricas que se obtienen son las siguientes:

In [16]:
print('Accuracy score: ' + str(accuracy_score(y_test, y_predicted)) + '\n')
print('F1 score: ' + str(f1_score(y_test, y_predicted)) + '\n')
print('Confusion matrix:\n')
print_confusion_matrix(y_test, y_predicted)

Accuracy score: 0.7937296865723886

F1 score: 0.3604006626133178

Confusion matrix:

     t/p      0     1 
        0 1002428 185117 
        1 95969 79193 
